In [1]:
%matplotlib inline

# Path management
import os
from pathlib import Path

# Get main project directory 
main_dir = str(Path(os.path.abspath('')).parents[0])
os.chdir(main_dir)
print('main dir:',main_dir)

main dir: /Users/pablo/OneDrive/data-science/github/isuelogit


In [2]:
# Internal modules
from src import isuelogit as isl

main dir: /Users/pablo/OneDrive/data-science/github/isuelogit


In [3]:
# External modules
import copy
import ast
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

In [4]:
# =============================================================================
# 2) NETWORKS FACTORY
# ============================================================================
network_name = 'Fresno'

# Estimation reporter
estimation_reporter = isl.writer.Reporter(foldername=network_name, seed = 2022)

# Reader of geospatial and spatio-temporal data
data_reader = isl.etl.DataReader(network_key=network_name)

# First Tuesday of October, 2020
data_reader.select_period(date = '2020-10-06', hour = 16)


Selected date is 2020-10-06, Tuesday at 16:00


In [5]:
# =============================================================================
# a) READ FRESNO LINK DATA
# =============================================================================

# Read nodes data
nodes_df = pd.read_csv(isl.dirs['input_folder'] + '/network-data/nodes/'  + 'fresno-nodes-data.csv')

# Read nodes spatiotemporal link data
links_df = pd.read_csv(
    isl.dirs['input_folder'] + 'network-data/links/' + str(data_reader.options['selected_date'])+ '-fresno-link-data.csv',
    converters={"link_key": ast.literal_eval,"pems_id": ast.literal_eval})


In [6]:
# =============================================================================
### Build network
# =============================================================================

# Create Network Generator
network_generator = isl.factory.NetworkGenerator()

A = network_generator.generate_adjacency_matrix(links_keys=list(links_df['link_key'].values))

fresno_network = \
    network_generator.build_fresno_network(A=A, links_df=links_df, nodes_df=nodes_df, network_name= network_name)


Creating Fresno network

Nodes: 1789, Links: 2413


In [7]:
# =============================================================================
# f) OD
# =============================================================================

# Reading OD matrix that was written internally
network_generator.read_OD(network=fresno_network, sparse=True)

# Average counts in 2020: 2213 and in 2021: 2152, which equates a scale factor of 0.97 in 2021
fresno_network.scale_OD(scale = 2152/2213)

Matrix Q (1789, 1789) read in 0.0[s] with sparse format
66266.34839999994 trips were loaded among 6970 o-d pairs
OD was scaled with factor 0.9724356077722549


In [8]:
# =============================================================================
# g) PATHS
# =============================================================================

# Create path generator
paths_generator = isl.factory.PathsGenerator()

# # Generate and Load paths in network
# paths_generator.load_k_shortest_paths(network = fresno_network, k=3)

paths_generator.read_paths(network=fresno_network, update_incidence_matrices=True)

26380 paths were read in 10.1[s] 100.0% 
26380 paths were loaded in the network
Updating incident matrices
Matrix D (2413, 26380) generated in 40.4[s]
Matrix M (6970, 26380) generated in 2.4[s] 
Matrix C (26380, 26380) generated in 9.0[s]


In [9]:
# =============================================================================
# d) LINK PERFORMANCE FUNCTIONS
# =============================================================================

bpr_parameters_df = pd.DataFrame({'link_key': links_df['link_key'],
                                  'alpha': links_df['alpha'],
                                  'beta': links_df['beta'],
                                  'tf': links_df['tf'],
                                  'k': pd.to_numeric(links_df['k'], errors='coerce', downcast='float')
                                  })

# Normalizate free flow travel time between 0 and 1
bpr_parameters_df['tf'] = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(np.array(bpr_parameters_df['tf']).reshape(-1, 1)))

fresno_network.set_bpr_functions(bprdata=bpr_parameters_df)


In [10]:
# =============================================================================
# 3c) FEATURE ENGINEERING
# =============================================================================
fresno_network.load_features_data(links_df, link_key = 'link_key')

# Spatio-temporal data must have read before
isl.etl.feature_engineering_fresno(links=fresno_network.links, network=fresno_network)
# ['low_inc', 'high_inc','no_incidents','no_bus_stops','no_intersections','tt_sd_adj','tt_reliability']

features_list = ['median_inc', 'intersections', 'incidents', 'bus_stops', 'median_age',
                 'tt_avg', 'tt_sd','tt_var', 'tt_cv',
                 'speed_ref_avg', 'speed_avg', 'speed_hist_avg','speed_sd','speed_hist_sd','speed_cv',
                 'tt_sd_adj','tt_reliability']


# Normalization of features to range [0,1]
linkdata = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(fresno_network.Z_data[features_list].values))
linkdata.columns = features_list
linkdata.insert(0, 'link_key', fresno_network.links_keys)

fresno_network.load_features_data(linkdata)

Features values of links with a type different than LWRLK were set to 0
New features: ['low_inc', 'high_inc', 'no_incidents', 'no_bus_stops', 'no_intersections', 'tt_sd_adj', 'tt_reliability']


In [11]:
# =============================================================================
# TRAFFIC COUNTS
# =============================================================================

# Read counts from csv
counts_df = pd.read_csv(isl.dirs['input_folder'] + '/network-data/links/' \
                            + str(data_reader.options['selected_date']) + '-fresno-link-counts' + '.csv',
                        converters={'link_key': ast.literal_eval})

counts = dict(zip(counts_df['link_key'].values, counts_df['counts'].values))

# Load counts
fresno_network.load_traffic_counts(counts=counts)


In [12]:
# =============================================================================
# 3) DESCRIPTIVE STATISTICS
# =============================================================================

In [13]:
# - Report link coverage

total_counts_observations = np.count_nonzero(~np.isnan(np.array(list(counts.values()))))

total_links = np.array(list(counts.values())).shape[0]

print('\nTotal link counts observations: ' + str(total_counts_observations))
print('Link coverage: ' + "{:.1%}".format(round(total_counts_observations / total_links, 4)))


Total link counts observations: 141
Link coverage: 5.8%


In [14]:
# - Networks topology
isl.descriptive_statistics.summary_table_networks([fresno_network])

,network,nodes,links,ods,paths
0,Fresno,1789,2413,6970,26380


In [15]:
# - Feature data 
summary_table_links_df = fresno_network.Z_data
estimation_reporter.write_table(df = summary_table_links_df, filename = 'links_data.csv', float_format = '%.3f')
summary_table_links_df

,link_type,alpha,beta,tf,k,observed,counts,capacity [veh],tt_ff [min],speed_ff[mi/hr],...,median_age,incidents,bus_stops,intersections,high_inc,low_inc,no_incidents,no_bus_stops,no_intersections,tt_reliability
0,LWRLK,0.15,4.0,0.098,1.800000e+03,0,NaN,1800.0,0.098,45,...,0.676087,0.748,0.435,0.30,1,0,0,0,0,0.403293
1,LWRLK,0.15,4.0,0.169,1.800000e+03,0,NaN,1800.0,0.169,50,...,0.747826,0.748,0.435,0.05,1,0,0,0,0,0.474777
2,LWRLK,0.15,4.0,0.396,2.400000e+03,0,NaN,2400.0,0.396,65,...,0.747826,0.748,0.435,0.05,1,0,0,0,0,0.474777
3,LWRLK,0.15,4.0,0.192,2.000000e+03,0,NaN,2000.0,0.192,25,...,0.747826,0.748,0.435,0.10,1,0,0,0,0,0.969572
4,LWRLK,0.15,4.0,0.105,1.800000e+03,0,NaN,1800.0,0.105,35,...,0.747826,0.748,0.435,0.10,1,0,0,0,0,0.477713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,PQULK,0.15,4.0,0.000,4.990000e+09,0,NaN,inf,0.000,99999,...,0.000000,0.000,0.000,0.00,0,0,0,0,0,0.000000
2409,PQULK,0.15,4.0,0.000,4.990000e+09,0,NaN,inf,0.000,99999,...,0.000000,0.000,0.000,0.00,0,0,0,0,0,0.000000
2410,LWRLK,0.15,4.0,0.045,1.800000e+03,0,NaN,1800.0,0.045,65,...,0.628261,1.000,0.435,0.45,1,0,0,0,0,0.511839
2411,LWRLK,0.15,4.0,0.107,2.400000e+03,0,NaN,2400.0,0.107,65,...,0.978261,0.500,0.435,0.70,0,1,0,0,0,0.501516


In [16]:
summary_table_links_df.describe()

,alpha,beta,tf,k,observed,counts,capacity [veh],tt_ff [min],speed_ff[mi/hr],inrix_id,...,median_age,incidents,bus_stops,intersections,high_inc,low_inc,no_incidents,no_bus_stops,no_intersections,tt_reliability
count,2.413000e+03,2413.0,2413.000000,2.413000e+03,2413.000000,141.000000,2413.0,2413.000000,2413.000000,1.840000e+03,...,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.0,2413.0,2413.0,2413.000000
mean,1.500000e-01,4.0,0.150823,1.439305e+09,0.058433,2152.176596,inf,0.150823,28872.664318,9.170812e+08,...,0.483083,0.522609,0.299036,0.228530,0.501450,0.210112,0.0,0.0,0.0,0.348203
std,2.776133e-17,0.0,0.172458,2.261116e+09,0.234610,844.686249,NaN,0.172458,45293.922006,6.053116e+08,...,0.323928,0.341960,0.213662,0.217516,0.500102,0.407472,0.0,0.0,0.0,0.232400
min,1.500000e-01,4.0,0.000000,1.800000e+03,0.000000,417.800000,1800.0,0.000000,15.000000,1.685461e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
25%,1.500000e-01,4.0,0.000000,1.800000e+03,0.000000,1625.000000,1800.0,0.000000,40.000000,4.416712e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
50%,1.500000e-01,4.0,0.121000,1.800000e+03,0.000000,2045.000000,1800.0,0.121000,45.000000,4.498919e+08,...,0.623913,0.748000,0.435000,0.200000,1.000000,0.000000,0.0,0.0,0.0,0.467086
75%,1.500000e-01,4.0,0.205000,4.990000e+09,0.000000,2510.000000,NaN,0.205000,99999.000000,1.626669e+09,...,0.686957,0.748000,0.435000,0.331000,1.000000,0.000000,0.0,0.0,0.0,0.487949
max,1.500000e-01,4.0,2.113000,4.990000e+09,1.000000,4564.000000,inf,2.113000,99999.000000,1.626774e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,1.000000


In [17]:
# =============================================================================
# 3) BILEVEL OPTIMIZATION IN CONGESTED NETWORK
# =============================================================================

utility_parameters = isl.estimation.Parameters(
    features_Y=['tt'],
    initial_values={'tt': 0},
)

utility_function = isl.estimation.UtilityFunction(utility_parameters)

equilibrator_norefined = isl.equilibrium.LUE_Equilibrator(
    network=fresno_network,
    paths_generator=paths_generator,
    utility_function=utility_function,
    # , uncongested_mode = True
    max_iters=100,
    method='fw',
    iters_fw=10,
    column_generation={'n_paths': 4,
                       'ods_coverage': 0.1,
                       'paths_selection': 4},
    path_size_correction=1
)

outer_optimizer_norefined = isl.estimation.OuterOptimizer(
    method='ngd',
    iters=1,
    eta=5e-1,
)

learner_norefined = isl.estimation.Learner(
    equilibrator=equilibrator_norefined,
    outer_optimizer=outer_optimizer_norefined,
    utility_function=utility_function,
    network=fresno_network,
    name = 'norefined'
)

equilibrator_refined = isl.equilibrium.LUE_Equilibrator(
    network=fresno_network,
    paths_generator=paths_generator,
    utility_function=utility_function,
    # , uncongested_mode = True
    max_iters=100,
    method='fw',
    iters_fw=10,
    path_size_correction=1
)

outer_optimizer_refined = isl.estimation.OuterOptimizer(
    method='lm',
    # method='ngd',
    iters=1,
    # eta=5e-1,
)

learner_refined = isl.estimation.Learner(
    network=fresno_network,
    equilibrator=equilibrator_refined,
    outer_optimizer=outer_optimizer_refined,
    utility_function=utility_function,
    name = 'refined'
)

In [18]:
# =============================================================================
# BENCHMARK PREDICTIONS
# =============================================================================

# Naive prediction using mean counts
mean_counts_prediction_loss, mean_count_benchmark_model \
    = isl.estimation.mean_count_prediction(x_bar=np.array(list(counts.values()))[:, np.newaxis])

print('\nObjective function under mean count prediction: ' + '{:,}'.format(round(mean_counts_prediction_loss, 1)))

# Naive prediction using uncongested network
equilikely_prediction_loss, predicted_counts_equilikely \
    = isl.estimation.loss_counts_equilikely_choices(
    network = fresno_network,
    equilibrator=equilibrator_refined,
    counts=fresno_network.counts_vector,
    utility_function=utility_function)

print('Objective function under equilikely route choices: ' + '{:,}'.format(round(equilikely_prediction_loss, 1)))


Objective function under mean count prediction: 99,889,346.8
Objective function under equilikely route choices: 281,458,809.7


### Estimation with travel time feature only

In [ ]:
print('\nStatistical Inference in no refined stage')

learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True)

theta_norefined = learning_results_norefined[best_iter_norefined]['theta']


Statistical Inference in no refined stage

Bilevel optimization for Fresno network 

Iteration : 1/10

Initial theta: {'tt': '0.0E+00'}

SUE via fw (max iters: 100)
Performed path size correction with factor 1

Performing path selection: dissimilarity_weight: 0 , paths per od: 4
Path selection with k=4 (New total paths: 26380)
Performed path size correction with factor 1

Equilibrium gaps: ['0E+00']
Initial Fisk Objective: -155,464.94
Final Fisk Objective: -155,464.94
Improvement Fisk Objective: 0.00%
Final gap: 0E+00. Acc. bound: 1E-04. Time: 523.0 [s]
Initial objective: 281,458,810
Initial RMSE: 1412.9
Initial Normalized RMSE: 0.656

Iteration : 2/10

Learning params via ngd (1 iters, eta = 5.0E-01)

theta: {'tt': '-5.0E-01'}------| 0.0% 
time: 463.9[s]

SUE via fw (max iters: 100)
Performed column generation with: 4 paths per od, 10.0% od coverage, demand sampling
136 paths added/replaced among 131 ods (total paths: 26516)
Performed path size correction with factor 1

Performing pa

In [ ]:
print('\nStatistical Inference in refined stage')

learner_refined.utility_function.parameters.initial_values = theta_norefined

learning_results_refined, inference_results_refined, best_iter_refined = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=2, alpha=0.05, link_report=False, iteration_report = True)

theta_refined = learning_results_refined[best_iter_refined]['theta']

In [ ]:
# Report 

estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined,
    theta_refined= theta_refined,
    best_objective_norefined = round(learning_results_norefined[best_iter_norefined]['objective'],1),
    best_objective_refined = round(learning_results_refined[best_iter_refined]['objective'],1),
)

# Summary with most relevant options, prediction error, initial parameters, etc
estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function)

# Write tables with results on learning and inference
estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined,
    results_refined=learning_results_refined,
    network = fresno_network,
    utility_function = utility_function)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined,
    results_refined=inference_results_refined,
    float_format = '%.3f')

In [ ]:
# =============================================================================
# 6) VISUALIZATIONS
# =============================================================================

# - Convergence

results_df = isl.descriptive_statistics \
    .get_loss_and_estimates_over_iterations(results_norefined=learning_results_norefined
                                            , results_refined=learning_results_refined)

fig = isl.visualization.Artist().convergence(
    results_norefined_df=results_df[results_df['stage'] == 'norefined'],
    results_refined_df=results_df[results_df['stage'] == 'refined'],
    filename='convergence_' + fresno_network.key,
    methods=[outer_optimizer_norefined.options['method'], outer_optimizer_refined.options['method']],
    folder = estimation_reporter.dirs['estimation_folder'],
    simulated_data = False
)

# - Distribution of errors across link counts

best_predicted_counts_norefined = np.array(list(learning_results_norefined[best_iter_norefined]['x'].values()))[:, np.newaxis]
best_predicted_counts_refined = np.array(list(learning_results_refined[best_iter_refined]['x'].values()))[:, np.newaxis]

fig, axs = plt.subplots(1, 2, sharey='all', tight_layout=True, figsize=(8, 6))

# We can set the number of bins with the `bins` kwarg
axs[0].hist(isl.estimation.error_by_link(observed_counts=np.array(list(counts.values()))[:, np.newaxis],
                                         predicted_counts=best_predicted_counts_norefined))
axs[1].hist(isl.estimation.error_by_link(observed_counts=np.array(list(counts.values()))[:, np.newaxis],
                                         predicted_counts=best_predicted_counts_refined))

for axi in [axs[0], axs[1]]:
    axi.tick_params(axis='x', labelsize=16)
    axi.tick_params(axis='y', labelsize=16)

plt.show()

fig.savefig(estimation_reporter.dirs['estimation_folder'] + '/' + 'distribution_predicted_error_counts.pdf',
            pad_inches=0.1, bbox_inches="tight")

### Estimation with all features

In [ ]:
utility_parameters_full_model = isl.estimation.Parameters(
    features_Y=['tt'],
    features_Z= ['tt_var', 'incidents', 'intersections', 'bus_stops', 'median_inc'],
    initial_values={'tt': 0},
)

utility_function_full_model = isl.estimation.UtilityFunction(utility_parameters_full_model)

learner_norefined.utility_function = utility_function_full_model
learner_refined.utility_function = utility_function_full_model

learner_norefined.utility_function.initial_values = theta_refined

In [ ]:
print('\nStatistical Inference in no refined stage')

learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, 
                                            link_report=False, iteration_report = True, 
                                            features_Z = utility_function_full_model.features_Z)

theta_norefined_full_model = learning_results_norefined[best_iter_norefined]['theta']

print('\nStatistical Inference in refined stage')

learner_refined.utility_function.parameters.initial_values = theta_norefined

learning_results_refined, inference_results_refined, best_iter_refined = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=2, alpha=0.05, 
                                          link_report=False, iteration_report = True)

theta_refined_full_model = learning_results_refined[best_iter_refined]['theta']

In [ ]:
# Write report

estimation_reporter = isl.writer.Reporter(foldername=network_name, seed = 2022)

estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_full_model,
    theta_refined= theta_refined_full_model,
    best_objective_norefined = round(learning_results_norefined[best_iter_norefined]['objective'],1),
    best_objective_refined = round(learning_results_refined[best_iter_refined]['objective'],1),
)

estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function_full_model)

estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined,
    results_refined=learning_results_refined,
    network = fresno_network,
    utility_function = utility_function_full_model)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined,
    results_refined=inference_results_refined,
    float_format = '%.3f')

### Estimation with features with expected sign

In [ ]:
utility_parameters_feature_selection_model = isl.estimation.Parameters(
    features_Y=['tt'],
    features_Z= ['tt_var', 'incidents', 'median_inc']
)

utility_function_feature_selection_model = isl.estimation.UtilityFunction(utility_parameters_feature_selection_model)

learner_norefined.utility_function = utility_function_feature_selection_model
learner_refined.utility_function = utility_function_feature_selection_model

learner_norefined.utility_function.initial_values = theta_refined_full_model

In [ ]:
print('\nStatistical Inference in no refined stage')

learning_results_norefined, inference_results_norefined, best_iter_norefined = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, 
                                            link_report=False, iteration_report = True, 
                                            features_Z = utility_function_full_model.features_Z)

theta_norefined_feature_selection_model = learning_results_norefined[best_iter_norefined]['theta']

print('\nStatistical Inference in refined stage')

learner_refined.utility_function.parameters.initial_values = theta_norefined

learning_results_refined, inference_results_refined, best_iter_refined = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=2, alpha=0.05, 
                                          link_report=False, iteration_report = True)

theta_refined_feature_selection_model = learning_results_refined[best_iter_refined]['theta']

In [ ]:
# Report

estimation_reporter = isl.writer.Reporter(foldername=network_name, seed = 2022)

estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_feature_selection_model,
    theta_refined= theta_refined_feature_selection_model,
    best_objective_norefined = round(learning_results_norefined[best_iter_norefined]['objective'],1),
    best_objective_refined = round(learning_results_refined[best_iter_refined]['objective'],1),
)

# Summary with most relevant options, prediction error, initial parameters, etc
estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function_feature_selection_model)

# Write tables with results on learning and inference
estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined,
    results_refined=learning_results_refined,
    network = fresno_network,
    utility_function = utility_function_feature_selection_model)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined,
    results_refined=inference_results_refined,
    float_format = '%.3f')